In [14]:
%load_ext autoreload
%autoreload 2
import os 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf 

from datetime import datetime
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array, array_to_img, load_img
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.python.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet152
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Input
from common_params import data_dir, img_dir
from common_plots import display_images
from data_load import get_train_val_set, apply_data_augmentation

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
imgsub_dir = os.path.join(data_dir,"30_classes_all")
IMG_SIZE = 224
batch_size = 32
image_size = (IMG_SIZE, IMG_SIZE)
input_shape = (IMG_SIZE, IMG_SIZE, 3) 

# Transfer learning using Xception 

In [5]:
train_ds, validation_ds = get_train_val_set(imgsub_dir, image_size, batch_size)
train_ds, validation_ds = apply_data_augmentation(train_ds, validation_ds)

Found 6164 files belonging to 30 classes.
Using 4932 files for training.
Using 1232 files for validation.
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [15]:
def get_xception_model():
    model = Sequential()
    model.add(Xception(include_top=False, pooling='avg', weights="imagenet"))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.8))
    model.add(Dense(120, activation='softmax'))
    model.layers[0].trainable = False
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
xmodel = get_xception_model()

In [ ]:
experiment_name = "xception-transfert-learning-dog-breed-classifier"
run_name = datetime.now().strftime("%Y%m%d_%H%M%S")
logdir = os.path.join(data_dir,"logs", experiment_name, run_name)
tb_callback = [tf.keras.callbacks.TensorBoard(log_dir=logdir, write_graph=True, histogram_freq=1)]

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=1)

xmodel.fit(x=train_ds, 
        epochs=5, 
        validation_data=validation_ds,
        callbacks=[tensorboard_callback])

# Transfer Learning with RestNet-152
tf.keras.applications.resnet.ResNet152 to load the ResNet-152 architecture with pre-trained ImageNet weights:

In [ ]:
def make_model() : 
    # Create an Input layer with the desired input shape


    inputs = keras.Input(shape=input_shape)
    # Load ResNet-152 model with pre-trained ImageNet weights
    resnet152_model = ResNet152(
        include_top=True,
        weights='imagenet',
        input_tensor=input_tensor,
        input_shape=input_shape,
        pooling=None,  # You can specify 'avg' or 'max' pooling here if needed
        classes=1000
    )

    x = resnet152_model(inputs, training=False)
    # Set the pooling layer
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    # Set the final layer with sigmoid activation function
    output = tf.keras.layers.Dense(num_classes, activation='sigmoid')(x)

    # Create the new model object
    model = tf.keras.Model(inputs=inputs , outputs=output)

    return model

In [ ]:
model = make_model()
model.summary()

## Model training 

In [ ]:
# Define hyperparameters and input data
learning_rate = 0.01
num_epochs = 5
input_shape = (224, 224, 3)

# Define names for tensorboard logging and mlflow
experiment_name = "dog-breed-classifier-ResNet152"
run_name = datetime.now().strftime("%Y%m%d_%H%M%S")

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.keras"),
]
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)
model.fit(
    train_ds,
    epochs=num_epochs,
    callbacks=callbacks,
    validation_data=validation_ds,
)

In [ ]:
img = keras.utils.load_img(
    os.path.join(img_dir,"n02085620-Chihuahua\\n02085620_7.jpg"), target_size=(IMG_SIZE, IMG_SIZE)
)
img_array = keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create batch axis

predictions = model.predict(img_array)
print(predictions)
# score = float(predictions[0])
# print(f"This image is {100 * (1 - score):.2f}% cat and {100 * score:.2f}% dog.")

In [ ]:

logdir = os.path.join(data_dir,"logs", experiment_name, run_name)
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir, write_graph=True, histogram_freq=1)

history = model.fit(
    augmented_train_dataset ,
    epochs=num_epochs,
    validation_data=validation_ds,
    callbacks=[tb_callback]
)

In [ ]:
# Set the experiment name and create an MLflow run
mlflow.set_experiment(experiment_name)
with mlflow.start_run(run_name = run_name) as mlflow_run:
    
    mlflow.set_experiment_tag("base_model", "MobileNet")
    mlflow.set_tag("optimizer", "keras.optimizers.Adam")
    mlflow.set_tag("loss", "sparse_categorical_crossentropy")

    mlflow.keras.log_model(model, "model")

    mlflow.log_param("learning_rate", learning_rate)
    mlflow.log_param("num_epochs", num_epochs)
    mlflow.log_param("batch_size", batch_size)
    mlflow.log_param("input_shape", input_shape)

    mlflow.log_metric("train_loss", history.history["loss"][-1])
    mlflow.log_metric("train_acc", history.history["accuracy"][-1])
    mlflow.log_metric("val_loss", history.history["val_loss"][-1])
    mlflow.log_metric("val_acc", history.history["val_accuracy"][-1])

    mlflow.log_artifact("model.png", "model_plot")

    mlflow_run_id = mlflow_run.info.run_id
    print("MLFlow Run ID: ", mlflow_run_id)